In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('port_data6km_ct.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'port_data6km_ct.csv'

In [ ]:
df

In [ ]:
import shapely.wkt
#df.coords = df.coords.apply(shapely.wkt.loads(str(coords)))

In [ ]:
df['new_berth'] = df.sort_values('t').groupby('sourcemmsi').navigationalstatus.diff()>0

In [ ]:
test_df = df[(df.navigationalstatus==5) & (df.new_berth)]

In [ ]:
from sklearn.cluster import DBSCAN
import geopandas as gpd

df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs='epsg:4326')

In [ ]:
df_geo[df_geo.speedoverground<0.3]

In [ ]:
from sklearn.datasets import make_blobs
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(
    n_samples=750, centers=centers, cluster_std=0.4, random_state=0
)


In [ ]:
def calculate_distance_and_speed(df):
    df = df.drop_duplicates(['id','t'])
    df = df.sort_values(['id', 't'])
    df['dt'] = (df.groupby('id')['t'].diff()).values  # Diff Time in seconds
    df['lon_prev'] = np.radians((df.groupby('id')['lon'].shift()).values)  
    df['lat_prev'] = np.radians((df.groupby('id')['lat'].shift()).values)
    df['lonrad'] = np.radians(df['lon'])
    df['latrad'] = np.radians(df['lat'])
    dlon = df.lon_prev - df.lonrad 
    dlat = df.lat_prev - df.latrad
    a = np.sin(dlat/2)**2 + np.cos(df.latrad) * np.cos(df.lat_prev) * np.sin(dlon/2)**2
    df['dist_m'] = (2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)))*6371000
    df['calculated_speed'] = df.dist_m/df.dt
    return df

In [ ]:
df_geo = calculate_distance_and_speed(df_geo.rename(columns={'sourcemmsi':'id'}))

In [ ]:
# remove all datapoints where speed > 0.5
df_geo = df_geo[df_geo.calculated_speed<0.5]

In [ ]:
df_geo

In [ ]:
df_moored = df[df.navigationalstatus==5]

In [ ]:
coords = df_moored[['lat', 'lon']].values

In [ ]:
'''
dbo = df_geo.to_crs(epsg=3857)
import contextily as cx
ax = dbo.plot(markersize=0.1, color='red')
cx.add_basemap(ax, crs=dbo.crs.to_string())
'''

In [ ]:
#db.set_geometry('geometry', inplace=True)

In [ ]:
#df_moored['weight'] = 
df_sizes = df_moored.groupby(['lon', 'lat']).size()

In [ ]:
'''
for index in df_sizes.index:
    mask = ((df_moored.lat == index[1]) & (df_moored.lon == index[0]))
    df_moored.loc[mask, 'weight'] = df_sizes[index]
'''

In [ ]:
df_moored.drop_duplicates(subset = ['lon', 'lat'], inplace =True)

In [ ]:
coords = test_df[['lat', 'lon']].values
#weights = df_moored.weight.values

In [ ]:
len(coords)

In [ ]:
coords = test_df[['lat', 'lon']].values
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))


In [ ]:
import matplotlib.pyplot as plt
db.labels_

In [ ]:
test_df['cluster'] = db.labels_

In [ ]:
import hdbscan
from sklearn.datasets import make_blobs

data, _ = make_blobs(1000)

clusterer = hdbscan.RobustSingleLinkage(cut=0.125, k=7)
cluster_labels = clusterer.fit_predict(data)
hierarchy = clusterer.cluster_hierarchy_
alt_labels = hierarchy.get_clusters(0.100, 5)
hierarchy.plot()


In [ ]:
coords = coords.astype('float32')

In [ ]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, min_samples=3, gen_min_span_tree=True, metric='haversine')
clusterer.fit(np.radians(coords))

In [ ]:
test_df

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import random
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
#x, y =circle_buffer.exterior.xy
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=1)
ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
for cluster in test_df.cluster.unique():
    ax.scatter(test_df[test_df.cluster==cluster].lon.values, test_df[test_df.cluster==cluster].lat.values, color=(random.random(),random.random(),random.random()), s=0.5, alpha=1)
#ax.plot(x, y)
#plt.ylim(top=48.52)
#plt.xlim(right=-4.63)
#plt.xlim(left=-4.3)

#plt.savefig('docked-ships-map.png')
plt.show()

In [ ]:

import shapely


def make_polygons(df):
    df.sort_values(by=['cluster'], ascending=[True], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['geometry'] = [shapely.geometry.Point(xy) for xy in zip(df['lon'], df['lat'])]
    anchs_clusters = pd.DataFrame()
    anchs_clusters_gdf = gpd.GeoDataFrame()
    gb = df.groupby('cluster')
    for y in gb.groups:
        df0 = gb.get_group(y).copy()
        point_collection = shapely.geometry.MultiPoint(list(df0['geometry']))
        # point_collection.envelope
        convex_hull_polygon = point_collection.convex_hull
        anchs_clusters = anchs_clusters.append(pd.DataFrame(data={'anchorage_id':[y],'geometry':[convex_hull_polygon]}))
        anchs_clusters_gdf = anchs_clusters_gdf.append(gpd.GeoDataFrame({'anchorage_id':[y],'geometry':[convex_hull_polygon]}))
    anchs_clusters.reset_index(inplace=True)
    anchs_clusters_gdf.reset_index(inplace=True)
    anchs_clusters_gdf.crs = 'epsg:4326'
    return anchs_clusters_gdf


In [ ]:
anchs_clusters = make_polygons(test_df)
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
for cluster in anchs_clusters.geometry[1:]:
    x, y = cluster.exterior.xy
    ax.plot(x,y)


In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt


 
llcrnrlat=48.35918890639375
llcrnrlon=-4.535391312214044
urcrnrlon=-4.417021242334729
urcrnrlat= 48.40895719576783
map = Basemap(projection='cyl',resolution='f', llcrnrlat=llcrnrlat, llcrnrlon = llcrnrlon, urcrnrlat=urcrnrlat, urcrnrlon= urcrnrlon)


map.drawcoastlines()

In [ ]:
for cluster in anchs_clusters.geometry[1:]:
    x, y = cluster.exterior.xy
    map.plot(x,y)
map.drawcoastlines()

In [ ]:
anchs_clusters.dtypes

In [ ]:
from st_visions.st_visualizer import st_visualizer
import st_visions.callbacks as callbacks
import st_visions.express as viz_express
import st_visions.geom_helper as viz_helper
def draw_clusters(clusters):
    st_viz = st_visualizer(allow_complex_geometries=False)
    st_viz.set_data(clusters)
    viz_express.plot_polygons_on_map(st_viz, polygon_type='patches')
    #st_viz.set_data(test_df_geo)
    #viz_express.plot_points_on_map(st_viz, color='red')
    st_viz.show_figures(notebook=True, notebook_url='localhost:8888')


In [ ]:
anchs_clusters_gdf = anchs_clusters.copy()
anchs_clusters_gdf.geometry = anchs_clusters_gdf.geometry.buffer(0.0005)
draw_clusters(anchs_clusters_gdf.drop(index=0))

In [ ]:
from sklearn.cluster import OPTICS


df_moored = df[df.navigationalstatus==5]
kms_per_radian = 6371.0088
epsilon = 0.2 / kms_per_radian
optics = OPTICS(min_samples=3, max_eps= epsilon, metric='haversine')

In [ ]:
test_df.groupby('cluster').sourcemmsi.unique()

In [ ]:
coords = np.radians(df_moored.head(10000)[['lat','lon']].values)

In [ ]:
oc = optics.fit(coords)

In [ ]:
len(oc.labels_)

In [ ]:
test_df.sort_values('t', inplace = True)

In [ ]:
df_geo['dlon'] = df_geo.groupby('id').lon.diff()
df_geo['dlat'] = df_geo.groupby('id').lat.diff()


In [ ]:
df_geo[df_geo.navigationalstatus==5].dlon.describe()

In [ ]:
df_geo[df_geo.navigationalstatus==1].dlon.describe()

In [ ]:
df_geo = pd.read_csv('port_data6km_ct.csv')
#df_geo = df_geo[(df_geo.calculated_speed>0) & (df_geo.calculated_speed<1)]

In [ ]:
df_geo.drop_duplicates(keep='first', inplace=True) # Drop duplicates
df_geo.drop_duplicates(subset=['sourcemmsi', 't'], inplace=True) 

In [ ]:
df_geo = calculate_distance_and_speed(df_geo.rename(columns= {'sourcemmsi':'id'}))

In [ ]:
df_geo[df_geo.calculated_speed>1]

In [ ]:
df_geo['speed_bool'] = (df_geo.calculated_speed<1)*1

In [ ]:
df_geo.sort_values(['id', 't'], inplace=True)

In [ ]:
consecutives = df_geo['speed_bool'].diff().ne(0).cumsum()

In [ ]:
df_geo['stop_num'] = consecutives

In [ ]:
sum((df_geo['t'].groupby(consecutives).agg(max)- df_geo['t'].groupby(consecutives).agg(min))>3600)

In [ ]:
df_geo['over_hour'] =df_geo.stop_num.map((df_geo['t'].groupby(consecutives).agg(max)- df_geo['t'].groupby(consecutives).agg(min))>3600)

In [ ]:
stop_groups = df_geo[df_geo.over_hour].groupby('stop_num')

In [ ]:
lon = stop_groups.lon.describe()['50%'].values
lat = stop_groups.lat.describe()['50%'].values

In [ ]:
coords = list(zip(lat,lon))

In [ ]:
coords = list(zip(lat,lon))
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [ ]:
db.labels_

In [ ]:
clusters = pd.DataFrame.from_dict({'lat': lat, 'lon':lon, 'cluster': db.labels_})

In [ ]:
cluster_polygons = make_polygons(clusters)
cluster_polygons.geometry = cluster_polygons.geometry.buffer(0.0005)

In [ ]:
draw_clusters(cluster_polygons.drop(index=0))

In [ ]:
s= gpd.read_file('../data_wrangling_test/data/[C1] World Port Index/WPI.shp')

In [ ]:
s.info()

In [ ]:
test_lines = gpd.read_file('test.shp')

In [ ]:
draw_clusters(anchs_clusters[(anchs_clusters.anchorage_id==8) | (anchs_clusters.anchorage_id==2)])

In [ ]:
anchs_clusters.geometry = anchs_clusters.geometry.buffer(0.0005)
def check_points_in_ports(anchs):
    for poly in anchs.geometry:
        intersect = False
        for line in test_lines.geometry:
            if poly.intersects(line):
                intersect= True
        assert intersect
        
        
check_points_in_ports(anchs_clusters)

In [ ]:
df.sort_values(['sourcemmsi', 't'],inplace=True)

In [ ]:
df['berth_num'] = df.new_berth.cumsum()

In [ ]:
berth_visits = df[df.navigationalstatus==5].groupby('berth_num')

In [ ]:
lon = berth_visits.lon.describe()['50%'].values
lat = berth_visits.lat.describe()['50%'].values
coords = list(zip(lat,lon))

In [ ]:
def train_model(coords):
    kms_per_radian = 6371.0088
    epsilon = 0.1 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    return db.labels_

In [ ]:
labels = train_model(coords)

In [ ]:
clusters = pd.DataFrame.from_dict({'lat': lat, 'lon':lon, 'cluster': labels})

In [ ]:
anchs = make_polygons(clusters)

In [ ]:
anchs_clusters_gdf

In [ ]:
anchs_clusters_gdf.geometry = anchs_clusters_gdf.geometry.buffer(0.0005)
draw_clusters(anchs_clusters_gdf[1:])


In [ ]:
clusters.groupby('cluster').size()

In [ ]:
anchs.geometry = anchs.geometry.buffer(0.0005)
check_points_in_ports(anchs)

In [ ]:
draw_clusters(test_lines)

In [ ]:
def intersection(point):
    for index, cluster in anchs_clusters_gdf[1:].iterrows():
        if point.within(cluster.geometry):
            return cluster.anchorage_id
    return -1

In [ ]:
anchs_clusters_gdf

In [ ]:
#df['cluster'] = df.geometry.apply(lambda x: intersection(x))


In [ ]:
df_geo = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs='epsg:4326')
df_geo['cluster'] = -1
for i, cluster in anchs_clusters_gdf[1:].iterrows():
    geom = cluster.geometry
    sindex = df_geo.sindex
    possible_matches_index = list(sindex.intersection(geom.bounds))
    possible_matches = df_geo.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(geom)]
    df_geo.loc[precise_matches.index, 'cluster'] = cluster.anchorage_id

In [ ]:
# Makes movement within possible overlapping clusters not registering as leaving from clusters
df_geo.loc[df_geo.cluster==-1, 'cluster'] = -999
df_geo.sort_values(['sourcemmsi','t'], inplace=True)
df_geo['enters_cluster']=df_geo.groupby('sourcemmsi').cluster.diff()>100
df_geo['leaves_cluster']=df_geo.groupby('sourcemmsi').cluster.diff()<-100

In [ ]:
anchs_clusters_gdf.geometry = anchs_clusters_gdf.geometry.buffer(0.0005)

In [ ]:
import pytz

df_geo['Date'] = pd.to_datetime(df_geo.t,unit='s')
df_geo['Date'] = df_geo['Date'].apply(lambda x: x.tz_localize('UTC').tz_convert('Europe/Paris'))

df_geo['clock'] = df_geo.Date.apply(lambda x: x.time())

df_geo['weekday'] = df_geo.Date.apply(lambda x: x.weekday())

In [ ]:
df_geo.clock

In [ ]:
df_geo.weekday

In [ ]:
df_geo[(df_geo.enters_cluster)]

In [ ]:
entry_times = df_geo[(df_geo.enters_cluster)][['clock', 'cluster']]
entry_times.clock = entry_times.clock.apply(lambda x: (x.hour*60 +x.minute)*60+x.second)
entry_times_desc = entry_times.groupby('cluster').clock.describe()
import datetime
for column in entry_times_desc.columns[1:]:
    entry_times_desc[column] = entry_times_desc[column].apply(lambda x: datetime.timedelta(seconds=x))

In [2]:
df_geo.sort_values(['sourcemmsi', 't'])

NameError: name 'df_geo' is not defined

In [ ]:
df_geo.loc[df_geo.cluster==-1, 'cluster'] = -999

In [ ]:
df_geo.sort_values(['sourcemmsi', 't'], inplace=True)

In [ ]:
df_geo.sort_values(['sourcemmsi','t'],inplace=True)
df_geo.leaves_cluster = df_geo.groupby('sourcemmsi').leaves_cluster.shift(-1)
df_geo.leaves_cluster.fillna(False, inplace=True)

In [ ]:
dep_times = df_geo[(df_geo.leaves_cluster)][['clock', 'cluster']]
dep_times.clock = dep_times.clock.apply(lambda x: (x.hour*60 +x.minute)*60+x.second)
dep_times_desc = dep_times.groupby('cluster').clock.describe()
import datetime
for column in dep_times_desc.columns[1:]:
    dep_times_desc[column] = dep_times_desc[column].apply(lambda x: datetime.timedelta(seconds=x))

In [ ]:
df_geo[df_geo.leaves_cluster]

In [ ]:
ships_per_cluster = df_geo.groupby('cluster').sourcemmsi.unique()

In [ ]:
ships_per_cluster.apply(lambda x: len(x))

In [ ]:
data_table = pd.DataFrame().from_dict({'Unique ships':[], 'Mean arrival time': [],'Mean departing time' : [], 'Mean time in cluster':[], 'Maximum ship length': [], 'Maximum ship draft': [], 'Ship types': []})

In [ ]:
data_table['Unique ships'] = ships_per_cluster.apply(lambda x: len(x))

In [ ]:
data_table['Mean arrival time'] = entry_times_desc['mean']
data_table['Mean departing time'] = dep_times_desc['mean']

In [ ]:
def td_to_str(td):
    """
    convert a timedelta object td to a string in HH:MM:SS format.
    """
    if pd.isnull(td):
        return 0
    hours, remainder = divmod(td.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    return f'{int(hours):02}:{int(minutes):02}:{int(seconds):02}'

In [ ]:
data_table['Mean arrival time'] = data_table['Mean arrival time'].apply(td_to_str)
data_table['Mean departing time'] = data_table['Mean departing time'].apply(td_to_str)

In [ ]:
df_geo.sort_values(['sourcemmsi','t'], inplace=True)
df_geo['change_in_cluster'] = (df_geo['enters_cluster'] | df_geo['leaves_cluster']).cumsum()

In [ ]:
visits = df_geo[df_geo.cluster>-1].groupby('change_in_cluster')

In [ ]:
times = visits.Date.max()-visits.Date.min()
clusters = visits.cluster.describe()['50%']

visits_df = pd.DataFrame().from_dict({'time': times.values, 'cluster': clusters.values})

In [ ]:
visits_df = visits_df[visits_df.time>datetime.timedelta(minutes=30)]

In [ ]:
data_table['Mean time in cluster'] = visits_df.groupby('cluster').time.describe()['mean']

In [ ]:
data_table

In [ ]:
static_data = pd.read_csv('nari_static.csv')

In [ ]:
static_data['length'] = static_data.tobow + static_data.tostern
static_data['beam'] = static_data.tostarboard + static_data.toport

In [ ]:
static_desc = static_data.groupby('sourcemmsi')[['length', 'beam', 'draught']].describe()

In [ ]:
static_df = pd.DataFrame().from_dict({'max_length': static_desc['length']['max'],'max_beam':static_desc['beam']['max'], 'max_draft' : static_desc['draught']['max']})

In [ ]:
static_df.loc[211286440]

In [ ]:
df_geo.groupby('cluster').sourcemmsi.unique()[1:]

In [ ]:
df_geo.groupby('cluster').sourcemmsi.unique()[1]

In [ ]:
def map_value(ind, col):
    return static_df.loc[ind][col]

map_func_len = lambda x: map_value(x, 'max_length')
map_func_beam = lambda x: map_value(x, 'max_beam')
map_func_draft = lambda x: map_value(x, 'max_draft')

max_len = [0]
max_beam = [0]
max_draft = [0]
for cluster in df_geo.groupby('cluster').sourcemmsi.unique()[1:]:
    max_len.append(max(map_func_len(cluster)))
    max_beam.append(max(map_func_beam(cluster)))
    max_draft.append(max(map_func_draft(cluster)))

In [ ]:
data_table['Maximum ship length'] = max_len
data_table['Maximum ship beam'] = max_beam
data_table['Maximum ship draft'] = max_draft

In [ ]:
data_table

In [ ]:
data_table['Ship types'] = (df_geo.groupby('cluster').vessel_type_name.value_counts()/df_geo.groupby('cluster').size()*100).drop(index=-999)

In [ ]:
data_table.drop(index=-999, inplace = True)

In [ ]:
data_table['Mean time in cluster'] = data_table['Mean time in cluster'].apply(lambda x: str(x).split(".")[0])

In [ ]:
df_geo[df_geo.enters_cluster]

In [ ]:
anchs_clusters_gdf.iloc[9:10]

In [ ]:
draw_clusters(anchs_clusters_gdf[1:])

In [ ]:
ship_percentage = (df_geo.groupby('cluster').vessel_type_name.value_counts()/df_geo.groupby('cluster').size()*100).drop(index=-999).reset_index(level=[1])

In [ ]:
df_geo[df_geo.leaves_cluster].groupby('cluster').size()

In [ ]:
no_z = df_geo[df_geo.cluster>-1]

data_table['Number of visits'] = no_z.groupby('cluster').change_in_cluster.unique().apply(lambda x:len(x))

In [ ]:
df_geo.groupby('cluster').sourcemmsi.unique().apply(lambda x:len(x))

In [ ]:
no_z = df_geo[df_geo.cluster>-1]
no_z.groupby('cluster').change_in_cluster.unique().apply(lambda x:len(x))

In [ ]:
c = list(data_table.columns)



#data_table = data_table.reindex(columns=column_names)

#data_table
data_table = data_table.reindex(columns= [c[8], c[0], c[1], c[2], c[3], c[4], c[5], c[7], c[6]])

In [ ]:
data_table

In [ ]:
ship_percentage.rename(columns= {0:'percentage'},inplace=True)

In [ ]:
dicts = {}
for ind, row in ship_percentage.iterrows():
    

In [ ]:
static_data